## 0. Установка путей

In [1]:
SCRIPT_PATH = 'Tensorflow/scripts/generate_tfrecord.py'
WORKSPACE_PATH = 'Tensorflow/workspace'
LABEL_MAP_PATH = 'Tensorflow/workspace/annotations/label_map.pbtxt'
TRAIN_RECORD_PATH = 'Tensorflow/workspace/annotations/train.record'
TEST_RECORD_PATH = 'Tensorflow/workspace/annotations/test.record'
MODEL_PATH = WORKSPACE_PATH + '/models'
APIMODEL_PATH = 'Tensorflow/models'
IMAGES_PATH = WORKSPACE_PATH + '/images'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
APP_TITLE = 'Обнаружение маски'

## 1. Создание карты меток

In [2]:
labels = [{'name': 'Mask', 'id': 1}, {'name': 'NoMask', 'id': 2}]

with open(LABEL_MAP_PATH, 'w') as f:
    for label in labels:
        f.write('item {\n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## 2. Создание файлов-записей обучения и теста Tensorflow

In [3]:
!python {SCRIPT_PATH} -x {IMAGES_PATH + '/train'} -l {LABEL_MAP_PATH} -o {TRAIN_RECORD_PATH}
!python {SCRIPT_PATH} -x {IMAGES_PATH + '/test'} -l {LABEL_MAP_PATH} -o {TEST_RECORD_PATH}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


## 3. Загрузка предварительно обученных моделей TensorFlow

In [4]:
!cd Tensorflow
# !git clone https://github.com/tensorflow/models

## 4. Загрузка и распаковка одноэтапной модели обнаружения объектов SSD-MobileNet V2

In [5]:
# # Загрузка тренированной модели
# wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
# # Перемещение архива
# !mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
# # Перемещение в папку с архивом
# !cd {PRETRAINED_MODEL_PATH} 
# # Распакова архива
# !tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

## 5. Копирование конфигурации модели в папку для обучения

In [6]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [7]:
!mkdir {'Tensorflow\workspace\models\\' + CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


## 6. Обновления конфигурации для трансферного обучения

In [8]:
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2
from object_detection.utils import config_util

In [9]:
CONFIG_FILE = MODEL_PATH + '/' + CUSTOM_MODEL_NAME + '/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_FILE)

In [10]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_FILE, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [11]:
# Список изменений в конфигурации
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = LABEL_MAP_PATH
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [TRAIN_RECORD_PATH]
pipeline_config.eval_input_reader[0].label_map_path = LABEL_MAP_PATH
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [TEST_RECORD_PATH]

In [12]:
# Запись изменений в файл конфигурации
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_FILE, 'wb') as f:
    f.write(config_text)

## 7. Тренировка модели

In [13]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=6000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))
# Для тренировки модели необходимо запустить выведенную команду в корневой директории проекта

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=6000


In [14]:
from object_detection.builders import model_builder
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utilz
import os

In [15]:
# Загрузка конфигурации и построение модели обнаружения
configs = config_util.get_configs_from_pipeline_file(CONFIG_FILE)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Восстановление точки сохранения
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-8')).expect_partial()

In [16]:
@tf.function
def detect(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## 9. Обнаружение маски в реальном времени

In [17]:
import cv2
import numpy as np

In [18]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)

In [21]:
camera = cv2.VideoCapture(0) # Видеозахват веб-камеры
width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret, frame = camera.read()
    image_np = np.array(frame)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utilz.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=2,
        line_thickness=2,
        min_score_thresh=0.4,
        agnostic_mode=False
    )

    cv2.imshow(APP_TITLE, cv2.resize(image_np_with_detections, (1024, 768)))

    if cv2.waitKey(1) == ord('q'):
        camera.release()
        cv2.destroyAllWindows()
        break